# 🧱 ETL - Limpieza y Transformación de Ventas de Notebooks
Este notebook crea la **capa Silver** a partir del dataset original almacenado en la capa Bronze de S3.

Pipeline: S3 (Bronze) → Limpieza (Python) → S3 (Silver)

In [1]:

# 📦 1. Importo las  librerías
import pandas as pd
import boto3
from io import StringIO

In [2]:
# ============================================================
# ⚙️ 2. Configuración de conexión
# ============================================================
bucket_name = 'notebooks-datalake-oalvarez'
bronze_key = 'bronze/notebooks_ventas.csv'
silver_key = 'silver/notebooks_ventas_clean.csv'

# Inicializar cliente S3 (usa las credenciales configuradas con aws configure)
s3 = boto3.client('s3')

In [3]:
# ============================================================
# 📥 3. Lectura de datos desde capa Bronze (S3)
# ============================================================
obj = s3.get_object(Bucket=bucket_name, Key=bronze_key)
df = pd.read_csv(obj['Body'])

print('✅ Dataset cargado desde Bronze:')
print(df.head())
print(f'\nRegistros totales: {len(df)}')

✅ Dataset cargado desde Bronze:
    año trimestre    marca  unidades_vendidas  ingreso_total continente
0  2022        Q3       HP            2838341     2318924597       Asia
1  2023        Q3  Samsung            5201221     5388464956     África
2  2023        Q3   Lenovo            2771890     2123267740    Oceanía
3  2023        Q2   Lenovo            2585840     3550358320     Europa
4  2022        Q1       HP            4377789     6028215453     Europa

Registros totales: 250


In [6]:
# ============================================================
# 🧹 4. Limpieza y normalización de datos
# ============================================================
# Eliminar duplicados y filas vacías
df = df.drop_duplicates()
df = df.dropna(subset=['marca', 'año', 'unidades_vendidas', 'ingreso_total'])

# Tipos de datos correctos
df['año'] = df['año'].astype(int)
df['unidades_vendidas'] = df['unidades_vendidas'].astype(int)
df['ingreso_total'] = df['ingreso_total'].astype(float)

# Normalización de texto
df['marca'] = df['marca'].str.title().str.strip()
df['continente'] = df['continente'].str.title().str.strip()

In [8]:
# ============================================================
# 🔍 5. Validaciones básicas
# ============================================================
print('\n📊 Información general del DataFrame:')
print(df.info())

print('\n🔢 Marcas únicas:')
print(df['marca'].value_counts())

print('\nAños disponibles:', sorted(df['año'].unique()))


📊 Información general del DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   año                250 non-null    int64  
 1   trimestre          250 non-null    object 
 2   marca              250 non-null    object 
 3   unidades_vendidas  250 non-null    int64  
 4   ingreso_total      250 non-null    float64
 5   continente         250 non-null    object 
dtypes: float64(1), int64(2), object(3)
memory usage: 11.8+ KB
None

🔢 Marcas únicas:
marca
Huawei       35
Asus         30
Msi          26
Samsung      26
Acer         26
Hp           24
Lenovo       22
Microsoft    22
Apple        20
Dell         19
Name: count, dtype: int64

Años disponibles: [np.int64(2020), np.int64(2021), np.int64(2022), np.int64(2023), np.int64(2024)]


In [9]:
# ============================================================
# 💾 6. Exportación a capa Silver (S3)
# ============================================================
csv_buffer = StringIO()
df.to_csv(csv_buffer, index=False)

s3.put_object(
    Bucket=bucket_name,
    Key=silver_key,
    Body=csv_buffer.getvalue()
)

print(f'\n✅ Archivo limpio cargado correctamente en S3 → {bucket_name}/{silver_key}')


✅ Archivo limpio cargado correctamente en S3 → notebooks-datalake-oalvarez/silver/notebooks_ventas_clean.csv
